<a href="https://colab.research.google.com/github/asifshaikat/hello-world/blob/main/qwen6b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Simple Chatbot for Qwen3-0.6B - Run this first
#@markdown This cell prepares the chatbot interface that will be displayed when you run the next cell.

import json
from IPython.display import display, HTML, Javascript

# Initialize the HTML code for the chat interface - use r""" for raw string to avoid unicode escape issues
HTML_CODE = r"""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Colab Chatbot GUI</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">

    <style>
        .chat-container { height: 450px; overflow-y: auto; background: #2d3748; border-radius: 8px; padding: 16px; }
        .chat-container::-webkit-scrollbar { width: 8px; }
        .chat-container::-webkit-scrollbar-track { background: #2d3748; border-radius: 10px; }
        .chat-container::-webkit-scrollbar-thumb { background: #4a5568; border-radius: 10px; }
        .chat-container::-webkit-scrollbar-thumb:hover { background: #718096; }
        .message { max-width: 80%; padding: 10px 15px; border-radius: 15px; margin-bottom: 10px; word-wrap: break-word; color: #e2e8f0; opacity: 0; transform: translateY(10px); animation: fadeIn 0.3s ease forwards; }
        .user-message { background-color: #319795; align-self: flex-end; border-bottom-right-radius: 5px; }
        .bot-message { background-color: #4a5568; align-self: flex-start; border-bottom-left-radius: 5px; }
        .thinking-indicator { font-style: italic; color: #a0aec0; }
        .typing-cursor::after { content: '▋'; display: inline-block; animation: blink 1s step-end infinite; margin-left: 2px; color: #a0aec0; }
        @keyframes fadeIn { to { opacity: 1; transform: translateY(0); } }
        @keyframes blink { from, to { opacity: 1 } 50% { opacity: 0 } }
        .code-toggle { font-size: 12px; cursor: pointer; color: #a0aec0; margin-top: 5px; }
        .code-panel { display: none; background: #1a202c; padding: 10px; border-radius: 5px; font-size: 12px; margin-top: 10px; max-height: 200px; overflow-y: auto; }
        .header-title { color: #f7fafc; font-weight: 600; }
    </style>
</head>
<body class="font-sans bg-gray-800 text-gray-200 pb-4">
    <div class="max-w-4xl mx-auto px-4">
        <header class="bg-gray-900 p-4 shadow-md flex-shrink-0 rounded-t-lg mt-2 border-b border-gray-700">
            <h1 class="text-xl header-title text-center">AI Chat Interface</h1>
        </header>

        <div id="chat-container" class="chat-container flex flex-col space-y-2">
            <div class="message bot-message" style="opacity: 1; transform: none;">Hello! I'm your AI assistant. How can I help you today? 😊</div>
        </div>

        <footer class="p-4 bg-gray-900 border-t border-gray-700 flex-shrink-0 rounded-b-lg">
            <div class="flex items-center space-x-2">
                <input type="text" id="user-input" class="flex-grow p-3 border border-gray-600 rounded-lg bg-gray-700 text-gray-200 placeholder-gray-500 focus:outline-none focus:ring-2 focus:ring-purple-500 focus:border-transparent" placeholder="Type your message...">
                <button id="multimodal-button" class="p-3 bg-teal-600 text-white rounded-lg hover:bg-teal-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-offset-gray-900 focus:ring-teal-500 transition duration-150 ease-in-out" title="Add File (Placeholder)">
                    <svg xmlns="http://www.w3.org/2000/svg" class="h-6 w-6" fill="none" viewBox="0 0 24 24" stroke="currentColor" stroke-width="2"><path stroke-linecap="round" stroke-linejoin="round" d="M15.172 7l-6.586 6.586a2 2 0 102.828 2.828l6.414-6.586a4 4 0 00-5.656-5.656l-6.415 6.585a6 6 0 108.486 8.486L20.5 13" /></svg>
                </button>
                <input type="file" id="file-input" class="hidden" accept="image/*,audio/*,video/*">
                <button id="send-button" class="p-3 bg-purple-600 text-white rounded-lg hover:bg-purple-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-offset-gray-900 focus:ring-purple-500 transition duration-150 ease-in-out" title="Send Message">
                    <svg xmlns="http://www.w3.org/2000/svg" class="h-6 w-6" fill="none" viewBox="0 0 24 24" stroke="currentColor" stroke-width="2"><path stroke-linecap="round" stroke-linejoin="round" d="M12 19l9 2-9-18-9 18 9-2zm0 0v-8" /></svg>
                </button>
            </div>
        </footer>

        <div class="mt-3 mb-3 text-center">
            <span class="code-toggle text-blue-400 hover:underline" onclick="toggleCodePanel()">Show/Hide Code</span>
            <div id="code-panel" class="code-panel">
                <p>Check the browser console (F12) for debugging information.</p>
            </div>
        </div>
    </div>

    <script>
        // DOM Elements
        const chatContainer = document.getElementById('chat-container');
        const userInput = document.getElementById('user-input');
        const sendButton = document.getElementById('send-button');
        const multimodalButton = document.getElementById('multimodal-button');
        const fileInput = document.getElementById('file-input');
        console.log("JS: DOM Elements ready.");

        // Global State
        let chatHistory = []; // Stores [[userMsg1, botMsg1], ...]
        console.log("JS: Initialized chatHistory:", JSON.stringify(chatHistory));

        // --- Core Functions ---
        function addMessage(text, sender) {
            console.log(`JS: addMessage - Sender: ${sender}, Text: "${text.substring(0, 60)}..."`);
            const messageDiv = document.createElement('div');
            messageDiv.classList.add('message', sender === 'user' ? 'user-message' : 'bot-message');
            messageDiv.textContent = text; // Use textContent for proper text handling
            chatContainer.appendChild(messageDiv);
            scrollToBottom();
            return messageDiv;
        }

        function showThinkingIndicator(text = "Thinking...") {
            console.log(`JS: showThinkingIndicator - Text: "${text}"`);
            const thinkingDiv = document.createElement('div');
            thinkingDiv.classList.add('message', 'bot-message', 'thinking-indicator-element'); // Add class to find later
            thinkingDiv.innerHTML = `<span class="thinking-indicator">${text}</span>`;
            chatContainer.appendChild(thinkingDiv);
            scrollToBottom();
            return thinkingDiv; // Return the element
        }

        function removeThinkingIndicator() {
            const indicator = chatContainer.querySelector('.thinking-indicator-element');
             if (indicator) {
                chatContainer.removeChild(indicator);
                console.log("JS: Removed thinking indicator.");
            }
        }

        function displayStreamedResponse(fullText, targetElement, delay = 25) {
             console.log(`JS: displayStreamedResponse - Target length: ${fullText.length}, Delay: ${delay}ms`);
            return new Promise(resolve => {
                const words = fullText.split(' '); // Split by spaces
                let currentText = ""; let wordIndex = 0;
                function typeWord() {
                    if (wordIndex < words.length) {
                        currentText += (wordIndex > 0 ? ' ' : '') + words[wordIndex];
                        targetElement.textContent = currentText;
                        scrollToBottom(); wordIndex++;
                        setTimeout(typeWord, delay);
                    } else {
                        targetElement.classList.remove('typing-cursor');
                        console.log("JS: displayStreamedResponse finished.");
                        resolve();
                    }
                }
                targetElement.classList.add('typing-cursor');
                targetElement.textContent = ''; // Clear previous content (like indicator)
                typeWord();
            });
        }

        async function sendMessage(text) {
            const trimmedText = text.trim();
            console.log(`JS: sendMessage called - Text: "${trimmedText}"`);
            if (!trimmedText) return;

            addMessage(trimmedText, 'user');
            userInput.value = ''; setUIBusy(true);
            const thinkingIndicator = showThinkingIndicator("Processing your message...");
            const currentHistory = chatHistory.slice();

            try {
                console.log("JS: Invoking backend 'notebook.handle_chat_message'...");
                console.log("JS: Sending history:", JSON.stringify(currentHistory));
                const result = await google.colab.kernel.invokeFunction(
                    'notebook.handle_chat_message', [trimmedText, currentHistory], {}
                );
                console.log("JS: Raw result from backend:", JSON.stringify(result, null, 2));

                removeThinkingIndicator();

                // Handle the JSON string with surrounding single quotes
                if (result && result.data && result.data['text/plain']) {
                    const responseJsonString = result.data['text/plain'];
                    console.log("JS: Received string:", responseJsonString);

                    try {
                        // Remove surrounding single quotes if they exist
                        let cleanJsonString = responseJsonString;
                        if (cleanJsonString.startsWith("'") && cleanJsonString.endsWith("'")) {
                            cleanJsonString = cleanJsonString.substring(1, cleanJsonString.length - 1);
                        }
                        console.log("JS: Cleaned JSON string:", cleanJsonString);

                        // Parse the cleaned JSON string
                        const responseData = JSON.parse(cleanJsonString);
                        console.log("JS: Parsed backend response data:", responseData);

                        const thinkingContent = responseData.thinking || "";
                        // Decode HTML entities in the final content
                        const finalContent = decodeHtmlEntities(responseData.final || "Error: Backend returned empty response.");
                        const isError = responseData.error || false;

                        // Create the bot message bubble to stream into
                        const botMessageElement = document.createElement('div');
                        botMessageElement.classList.add('message', 'bot-message');
                        chatContainer.appendChild(botMessageElement);
                        scrollToBottom();
                        console.log("JS: Added empty bot message bubble for streaming.");

                        await displayStreamedResponse(finalContent, botMessageElement);

                        if (!isError) {
                            chatHistory.push([trimmedText, finalContent]);
                            console.log("JS: Updated chatHistory:", JSON.stringify(chatHistory));
                        } else {
                            console.warn("JS: Not adding error response to chat history.");
                        }

                    } catch (parseError) {
                        console.error("JS: Error parsing JSON response from backend:", parseError);
                        console.error("JS: Received string was:", responseJsonString);
                        addMessage("Error: Could not parse response from backend.", 'bot');
                    }
                } else {
                    console.error("JS: Error - Invalid response structure:", result);
                    addMessage("Error: Received unexpected response structure from backend.", 'bot');
                }

            } catch (error) {
                console.error("JS: Error calling Colab kernel function:", error);
                removeThinkingIndicator();
                addMessage(`Error communicating with the backend. (${error.message || error})`, 'bot');
            } finally {
                setUIBusy(false);
            }
        }

        // Function to decode HTML entities and properly handle Unicode
        function decodeHtmlEntities(text) {
            const textarea = document.createElement('textarea');
            textarea.innerHTML = text;

            // Get the decoded text
            let decodedText = textarea.value;

            // Fix any escaped Unicode sequences (convert \uXXXX to actual characters)
            decodedText = decodedText.replace(/\\u([a-fA-F0-9]{4})/g, function(match, code) {
                return String.fromCharCode(parseInt(code, 16));
            });

            return decodedText;
        }

        async function handleFileSelect(event) {
            console.log("JS: handleFileSelect triggered.");
            const files = event.target.files;
            if (files.length === 0) return;
            const file = files[0];
            console.log(`JS: File selected - Name: ${file.name}, Type: ${file.type}, Size: ${file.size} bytes`);
            addMessage(`Selected file: ${file.name} (${Math.round(file.size / 1024)} KB)`, 'user');

            const thinkingIndicator = showThinkingIndicator("Processing file...");
            await new Promise(resolve => setTimeout(resolve, 1500));
            addMessage(`File "${file.name}" processing is not implemented yet.`, 'bot');
            removeThinkingIndicator();
            fileInput.value = '';
            console.log("JS: Placeholder file handling complete.");
        }

        // --- Utility Functions ---
        function scrollToBottom() { chatContainer.scrollTop = chatContainer.scrollHeight; }
        function setUIBusy(isBusy) {
             userInput.disabled = isBusy; sendButton.disabled = isBusy;
             multimodalButton.disabled = isBusy;
             if (!isBusy) { userInput.focus(); }
             console.log(`JS: UI ${isBusy ? 'disabled' : 'enabled'}.`);
        }

        function toggleCodePanel() {
            const codePanel = document.getElementById('code-panel');
            codePanel.style.display = codePanel.style.display === 'none' ? 'block' : 'none';
        }

        // --- Event Listeners ---
        console.log("JS: Adding event listeners...");
        sendButton.addEventListener('click', () => sendMessage(userInput.value));
        userInput.addEventListener('keypress', (event) => {
            if (event.key === 'Enter' && !event.shiftKey) { event.preventDefault(); sendMessage(userInput.value); }
        });
        multimodalButton.addEventListener('click', () => { fileInput.click(); });
        fileInput.addEventListener('change', handleFileSelect);
        console.log("JS: Event listeners added.");

        // --- Initial Setup ---
        console.log("JS: Chatbot GUI Initialized.");
        if (typeof google !== 'undefined' && google.colab && google.colab.kernel) {
            console.log("JS: google.colab.kernel is available.");
        } else {
            console.warn("JS: google.colab.kernel not detected. Running outside Colab?");
             addMessage("Warning: Cannot connect to Colab backend.", 'bot');
        }
        userInput.focus(); scrollToBottom();
        window.toggleCodePanel = toggleCodePanel;
    </script>

</body>
</html>
"""

# Define the handler function that will be used by the frontend
def handle_chat_message(user_message, chat_history):
    """Process a chat message and return a response"""
    try:
        # This is where you would connect to your LLM API
        # For this example, we'll just create a simple echo response
        thinking = f"<think>\nProcessing user message: '{user_message}'\nFormulating an appropriate response..."

        # Simple response for demonstration
        if "who are you" in user_message.lower():
            final_response = "I'm an AI assistant designed to help you with questions and tasks. I can provide information, answer questions, and assist with various activities. Let me know how I can help! 😊"
        elif "ai" in user_message.lower() and "human" in user_message.lower():
            final_response = "AI (Artificial Intelligence) and humans are not competitors but partners in the same world. AI excels at tasks like learning, problem-solving, and data analysis, while humans bring creativity, emotional intelligence, and personal experiences. Together, they complement each other in driving innovation and progress. 🤝"
        else:
            final_response = f"You said: '{user_message}'. This is a placeholder response. In a real implementation, this would connect to an LLM API. 👍"

        # Return the response as a JSON string (this will be parsed by the frontend)
        return json.dumps({"thinking": thinking, "final": final_response})
    except Exception as e:
        # Return error information
        return json.dumps({"error": True, "final": f"Error processing message: {str(e)}"})

# Register the handler function so it can be called from JavaScript
from google.colab import output
output.register_callback('handle_chat_message', handle_chat_message)

print("✅ Chat interface setup complete! Run the next cell to display the interface.")

✅ Chat interface setup complete! Run the next cell to display the interface.


In [ ]:
#@title Chat Interface
#@markdown This cell displays the chat UI. Make sure you've run Cell 1 first.

from IPython.display import display, HTML

# Check if the HTML_CODE variable exists from the previous cell run
if 'HTML_CODE' in locals():

    display(HTML(HTML_CODE))

else:
    print("❌ Error: HTML_CODE variable not found.")
    print("Please make sure you have successfully run Cell 1 first.")

Backend received prompt: 'tell me a joke about abngladesh...'
Generating response...
No thinking token found, using full output as final.
Thinking part: '...'
Final part: '<think>
Okay, the user wants a joke about Banglade...'
Generation took: 25.85 seconds
